### Notebook to develop and test the Pomodoro App

In [15]:
import os
import time
import platform
import threading
import tkinter as tk
from PIL import Image, ImageTk  
from playsound import playsound
from tkinter import font, messagebox
from config import *

class PomodoroApp:
    # 🍅 The full app is encapsulated by this class 🍅 
    # This is a Castlevania themed Pomodoro app with costumizible work and break timers and cycles
    # App can be stopped and restarted at any point or Reset and change timer/cycle values
    
    def __init__(self, root):
        # Get the directory of the current file
        if '__file__' in locals():
            self.base_path = os.path.dirname(os.path.abspath(__file__))
        else:
            # Fallback: Use current working directory if __file__ is not defined
            self.base_path = os.getcwd()

        #  Defining instance variables
        self.root = root
        self.root.title("My 🍅 Timer") # main title
        self.is_running = False # flag for running the timer, default is False
        self.work_mins = 33 # default work duration in minutes
        self.break_mins = 7 # default break duration in minutes
        self.time_left = self.work_mins * 60 # remaining time converted to seconds
        self.is_working = True # flag to distinguish working timer and break timer
        self.cycles = 2 # default number of cycles
        self.cycle_number = 0 # current cycle number
        self.timer_win_size = timer_win_size

        # Load the PNG background
        bg_vania = "images/background.png"
        bg_ACOTAR = "images/Velaris_bg.jpeg"
        self.background_image = Image.open(self.rel_path_to(bg_vania))  # load PNG file
        self.background_image = self.background_image.resize((400, 300), Image.LANCZOS)  # resize to fit the window
        self.background_photo = ImageTk.PhotoImage(self.background_image) # load to image
        # Create a label for the background
        self.background_label = tk.Label(self.root, image=self.background_photo)
        self.background_label.place(relwidth=1, relheight=1)  # fill the window with the background image
        
        # Fonts used for text, input numbers, and timer
        font_vania = 'Angel wish'
        font_ACOTAR = "The Centurion"
        font_text = font.Font(family=font_vania, size=font_text_size, weight='bold')
        font_button = font.Font(family=font_vania, size=font_butt_size, weight='bold')
        font_number = font.Font(family="Cloister Black", size=font_number_size)
        self.font_timer = font.Font(family="The Centurion", size=font_timer_size, weight="bold")

        # Defining args for positioning
        self.work_label_pack = ['w', 10, (15,2)] # args for position and padding of work time label
        self.pack_args = ['w', 10, 2] # args for position and padding of remaining labels
        
        # Input for Work time
        self.work_label = tk.Label(self.root, text=' Work Time ', font=font_text , fg=text_color, bg=text_bg_color) # text label
        self.work_label.pack(anchor=self.work_label_pack[0], padx=self.work_label_pack[1], pady=self.work_label_pack[2])
        self.work_entry = tk.Entry(self.root, font=font_number, width=3, justify='center', highlightthickness=1, highlightcolor='white',
                                   fg=text_color, bg=entry_bg_color, insertbackground=text_color) # input label
        self.work_entry.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.work_entry.insert(0, str(self.work_mins)) # inserts default value
        
        # Input for break time
        self.break_label = tk.Label(self.root, text=' Break Time ', font=font_text, bd=0, fg=text_color, bg=text_bg_color) # text label
        self.break_label.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.break_entry = tk.Entry(self.root, font=font_number, width=3, justify='center', highlightthickness=1, highlightcolor='white',
                                   fg=text_color, bg=entry_bg_color, insertbackground=text_color) # input label
        self.break_entry.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.break_entry.insert(0, str(self.break_mins)) # default value
        
        # Input for cycle
        self.cycle_label = tk.Label(self.root, text='Cycles', font=font_text, bd=0, fg=text_color, bg=text_bg_color) # text label
        self.cycle_label.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.cycle_entry = tk.Entry(self.root, font=font_number, width=2, justify='center', highlightthickness=1, highlightcolor='white',
                                   fg=text_color, bg=entry_bg_color, insertbackground=text_color) # input label
        self.cycle_entry.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.cycle_entry.insert(0, str(self.cycles)) # default value
        
        # Timer display label using a custom background
        self.timer_bg_og = Image.open(self.rel_path_to("images/bg1.png"))
        self.timer_bg_og_resize = self.timer_bg_og.resize((170, 95), Image.LANCZOS) 
        self.timer_bg = ImageTk.PhotoImage(self.timer_bg_og_resize)
        self.timer_label = tk.Label(self.root, image=self.timer_bg, text=self.format_time(self.time_left), font=self.font_timer, fg=text_color,
                                    compound='center', bd=0, padx=0, pady=0) # adding background to the label
        
        # Button label using a custom background
        self.buttons_bg_og = Image.open(self.rel_path_to("images/bg1.png"))
        self.buttons_bg_og_resize = self.buttons_bg_og.resize(butt_img_size, Image.LANCZOS)
        self.buttons_bg = ImageTk.PhotoImage(self.buttons_bg_og_resize)
        self.button_label = tk.Label(self.root, image=self.buttons_bg, bd=0, padx=0, pady=0)
        self.button_label.pack(side="bottom", fill='x', padx=10, pady=10)

        # Start, Stop and Reset buttons inside the Button label
        self.start_button = tk.Button(self.button_label, text='Start', font=font_button, command=self.start_timer, bd=1, fg=text_color, 
                                      activeforeground=text_color, bg=butt_bg_color, activebackground=butt_bg_color_act, width=butt_width, 
                                      height=butt_height, padx=0, pady=0, highlightthickness=0, highlightbackground=butt_bg_color)
        self.start_button.pack(side='left', padx=Lbutt_padx, pady=butt_pady)
        
        self.stop_button = tk.Button(self.button_label, text='Stop', font=font_button, command=self.stop_timer, bd=1, fg=text_color, 
                                      activeforeground=text_color, bg=butt_bg_color, activebackground=butt_bg_color_act, width=butt_width, 
                                      height=butt_height, padx=0, pady=0, highlightthickness=0,highlightbackground=butt_bg_color)
        self.stop_button.pack(side='left', padx=Mbutt_padx, pady=butt_pady)
        
        self.reset_button = tk.Button(self.button_label, text='Reset', font=font_button, command=self.reset_timer, bd=1, fg=text_color, 
                                      activeforeground=text_color, bg=butt_bg_color, activebackground=butt_bg_color_act, width=butt_width, 
                                      height=butt_height, padx=0, pady=0, highlightthickness=0, highlightbackground=butt_bg_color)
        self.reset_button.pack(side='right', padx=Rbutt_padx, pady=butt_pady)
        
        # Set starting window size (width x height)
        self.root.geometry("")  # Auto adjusts window size according the the packed labels
        self.root.resizable(False,False) # Stop user from resizing window

    def start_timer(self):
        # Get user input for the timers
        try:
            self.work_mins = float(self.work_entry.get())
            self.break_mins = float(self.break_entry.get())
            self.cycles = int(self.cycle_entry.get())
            self.time_left = self.work_mins * 60
            # Hide input fields
            self.work_label.pack_forget()
            self.work_entry.pack_forget()
            self.break_label.pack_forget()
            self.break_entry.pack_forget()
            self.cycle_label.pack_forget()
            self.cycle_entry.pack_forget()
            # Show timer label
            self.timer_label.place(relx=0.5, rely=0.5, anchor='center')
            self.root.geometry(self.timer_win_size) # resizing window
            # Start running the timer
            self.is_running = True # set the running flag
            self.root.title(f"Work Timer 🍅") # change title
            self.play_sound_threaded(self.rel_path_to('sounds/SoulSteal.wav')) # sound for start of work timer
            self.run_timer()
        except ValueError:
            messagebox.showerror("Invalid input", "Enter int or decimal for the timers and int for cycles, ex.: 1 - 0.5 - 1") 
        if self.work_mins <= 0 or self.break_mins <= 0 or self.cycles <= 0:
            messagebox.showerror("Invalid input", "Timers and cycles must be greater than zero.")
        return
    
    def run_timer(self):
        if  self.is_running:
            if self.time_left > 0:
                self.time_left -= 1 # decrease time left value
                self.timer_label['text'] = self.format_time(self.time_left) # update timer label
                self.root.after(1000, self.run_timer) # call run_timer again after 1 second
            else:
                self.switch_timer() # if time left is 0 switch the other timer
    
    def switch_timer(self):
        if self.is_working: # if previous timer was work change to break
            self.root.title(f"Break Timer 🍅")
            self.time_left = self.break_mins * 60 # set timer for the break
            self.timer_label['text'] = self.format_time(self.time_left)
        else: # if previous timer was break change to work
            self.root.title(f"Work Timer 🍅")
            self.time_left = self.work_mins * 60 # set timer for work
            self.cycle_number += 1 # increment cycle number
            
        self.is_working = not self.is_working # change working flag to opposito, to alternate timers
        
        if self.cycle_number < self.cycles: # if number of cycles less than desired number continue running the timer
            if self.is_working:
                self.play_sound_threaded(self.rel_path_to('sounds/SoulSteal.wav')) # play sound for work timer
            else:
                self.play_sound_threaded(self.rel_path_to('sounds/DarkMeta.wav')) # play sound for break timer
            self.run_timer()
        else:
            self.root.geometry("300x200")
            self.play_sound_threaded(self.rel_path_to('sounds/Impressive.wav'))
            self.root.title(f"The end 🍅") # change title to end title
            self.timer_bg_og_resize = self.timer_bg_og.resize((260, 100), Image.LANCZOS) 
            self.timer_bg = ImageTk.PhotoImage(self.timer_bg_og_resize)
            self.timer_label.config(image=self.timer_bg, text="🍅 Finished! 🍅", font=self.font_timer)
            self.is_running = False 
    
    def stop_timer(self):
        self.play_sound_threaded(self.rel_path_to('sounds/What.wav')) # sound for stop button
        self.is_running = False # restarts the timer as well by setting flag to false
    
    def reset_timer(self):
        self.play_sound_threaded(self.rel_path_to('sounds/AsYouWish.wav'))
        self.root.title(f" My 🍅 Timer ")
        # Show input fields again
        self.work_label.pack(anchor=self.work_label_pack[0], padx=self.work_label_pack[1], pady=self.work_label_pack[2])
        self.work_entry.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.break_label.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.break_entry.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.cycle_label.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        self.cycle_entry.pack(anchor=self.pack_args[0], padx=self.pack_args[1], pady=self.pack_args[2])
        # Hide timer
        self.timer_label.place_forget() 
        self.root.geometry("") 
        self.is_running = False
        self.time_left = self.work_mins * 60 # reset to default
        self.cycle_number = 0 # reset to default
        self.timer_label['text'] = self.format_time(self.time_left) # reset to default
       
    def format_time(self, time_in_seconds): # formats time in minutes to int number of seconds in a string
        minutes, seconds = divmod(time_in_seconds, 60)
        return f"{int(minutes):02}:{int(seconds):02}"

    def play_sound_threaded(self, sound_file): # plays sound in a thread to keep GUI active
        threading.Thread(target=playsound, args=(sound_file,)).start()
    
    def rel_path_to(self, file):
            return os.path.join(self.base_path, file)

In [16]:
root = tk.Tk() # creates tkinter object
app = PomodoroApp(root)
root.mainloop() # starts the App

A Read me Preview - old 

# 🍅 Pomodoro App (Python) 🍅

A simple pomodoro app using Python 3.9.2 and pip 24.2, it's also already packaged for MacOS  

What's a pomodoro? --> https://en.wikipedia.org/wiki/Pomodoro_Technique


## How to Use
clone the repo or download the zip from the Github page (hint: green "Code" button)

#### With MacOs 

- via App file:  
The app is already packaged for MacOs, this means after cloning (or unzip) the repo, can go to the folder "*/dist*" and double-click the pomodoro app icon 🍅.  
The app works for my current system: macOS 11.6.4 (20G417), other systems could give an issue.

#### With Python
[Install python](https://www.python.org)
- via python file:  
Can also start the app directly from the python file (this requires using the terminal and having correct python and pip versions).
```sh
cd path/to/repo/Pomodoro_App # move to the directory with the app
python3 -m venv venv # create a virtual environment for this app's libraries
source venv/bin/activate # activate the venv just created
pip install -r requirements.txt # install all the requirements
python pomodoro.py # launch the app
```
Should work with the previous code, if there are permissions issue can try adding this to give executable permissions:
```sh
...
chmod +x pomodoro.py # give permisison
python pomodoro.py # launch the app
```

- via jupyter notebook:  
There is also a jupyter notebook in the repo used for development and testing. Can also launch the app by running the first 2 cells:
```sh
cd path/to/repo/Pomodoro_App # move to the directory with the app
python3 -m venv venv # create a venv for this app libraries
source venv/bin/activate # activate the venv just created
pip install -r requirements.txt # install all the requirements
python -m ipykernel install --user --name=venv --display-name "Pomodoro Venv" # add the venv to Jupyter as a kernel
jupyter lab # open jupyter lab
```  
After the Jupyter lab opens can open the file "Pomodoro App testing.ipynb" and run the first cell with code by clicking on the cell and then Shift+Enter.

### If everything works:
- Enjoy!  (˵ •̀ ᴗ - ˵ ) ✧
### else:
- Consider contacting me: goncalo.ag.felicio@gmail.com

---

# To Do's 
( somewhere, sometime in the distant future.. or by the devine grace of a gentle soul that contributes their pen  
૮꒰ ˶• ༝ •˶꒱ა ♡ )

- Package the app to also work on windows,
- Open app by clicking on the terminal executable inside /dist (opens much faster),
- Auto adjust the background image to always match size of the window.

# Final Notes

When packaging the app i first tried with py2app, this did not work, even troubleshooting and trying a lot of fixes did not work in creating a launchable app icon, even though it worked by launching from the terminal executable. Don't know if it has to do with my MacOs version or was doing something else wrong. The build time was also much longer than with pyinstaller, but with pyinstaller i had to adjust the pathing to the resources and add the images and sounds folder manually to the app folder.

It seems that tkinter simply must have the fonts installed system wide so can load them from a path. When Trying to install the fonts through python to automate font loading, its just not practical as it requires user permission on the python file or bash script to make the fonts system wide available. In the end best for the user to simply install them at the start.